In [3]:
# ref:https://www.physionet.org/content/eegmmidb/1.0.0/
#imagine - Task2 & Task4 Correspond R04、R06、R08、R10、R12、R14

from glob import glob
import mne
from tqdm import tqdm
import pandas as pd
import numpy as np

imagineTasks = ['R04','R06','R08','R10','R12','R14']
allEdf = glob('../dataset/files/*/*.edf')
filteredEdf = [edf for edf in allEdf if any(sub in edf for sub in imagineTasks)]

In [4]:
dataset = pd.DataFrame()
targets = []
Tasks = []
for edf in filteredEdf:
    edfSample = mne.io.read_raw_edf(edf)
    events, event_id = mne.events_from_annotations(edfSample)
    
    eventStart = events[:,0]
    labels = events[:,2]
    channel_names = edfSample.info.ch_names
    #only use Fp1 signal
    Fp1idx = channel_names.index('Fp1.')
    
    Fp1signal = edfSample.get_data()[Fp1idx,:]
    eventsSignal = []
    eventTimes = events[:,0]
    for i in range(len(eventTimes)):
        start = eventTimes[i]
        if i +1 == len(eventTimes):
            eventSignal = Fp1signal[start:]
            eventsSignal.append(eventSignal)
            break
        end = eventTimes[i+1]
        eventSignal = Fp1signal[start:end]
        eventsSignal.append(eventSignal[:640])
    
    dataset = pd.concat((dataset, pd.DataFrame(eventsSignal)), axis=0)
    targets.extend(labels)
        # record what action
    Tasks.extend([edf.split('/')[-1].split('.')[0]] * len(labels))

Extracting EDF parameters from /Users/junyangding/EEGRobot/CompletedEEGCode/dataset/files/S038/S038R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/junyangding/EEGRobot/CompletedEEGCode/dataset/files/S038/S038R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/junyangding/EEGRobot/CompletedEEGCode/dataset/files/S038/S038R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/junyangding/EEGRobot/CompletedEEGCode/dataset/files/S038/S038R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters fr

/var/folders/pj/n76y7w2n1b18fq36p9y416_h0000gn/T/ipykernel_13088/3817734525.py:5: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  edfSample = mne.io.read_raw_edf(edf)
/var/folders/pj/n76y7w2n1b18fq36p9y416_h0000gn/T/ipykernel_13088/3817734525.py:5: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  edfSample = mne.io.read_raw_edf(edf)
/var/folders/pj/n76y7w2n1b18fq36p9y416_h0000gn/T/ipykernel_13088/3817734525.py:5: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  edfSample = mne.io.read_raw_edf(edf)
/var/folders/pj/n76y7w2n1b18fq36p9y416_h0000gn/T/ipykernel_13088/3817734525.py:5: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  edfSample = mne.io.read_raw_edf(edf)
/var/folders/pj/n76y7w2n1b18fq36p9y416_h0000gn/T/ipykernel_13088/3817734525.py:5: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  edfSample = 

Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/junyangding/EEGRobot/CompletedEEGCode/dataset/files/S101/S101R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/junyangding/EEGRobot/CompletedEEGCode/dataset/files/S101/S101R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/junyangding/EEGRobot/CompletedEEGCode/dataset/files/S101/S101R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/junyangding/EEGRobot/CompletedEEGCode/dataset/files/S101/S101R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations description

In [5]:
dataset = dataset.iloc[:,:640].reset_index(drop=True)
dataset['target'] = targets
dataset['Task'] = Tasks
dataset = dataset.dropna()

In [6]:
# R04 T1- left fist  R04 T2- right fist
# R06 T1 - double fists  R06 T2 - double feet 
# R08 T1 - left fist  R08 T2 - right fist 
# R10 T1 - double fists  R10 T2 - double feet 
# R12 T1 - left fist  R12 T2 - right fist
# R14 T1 - double fists  R14 T2 - double feet 
# left 10
# right 20
# double fists 30
#double feet 40

dataset.loc[dataset.target==1,'target'] = 0
dataset.loc[(dataset.Task.str.contains('R04')) & (dataset.target==2),'target'] = 10
dataset.loc[(dataset.Task.str.contains('R04')) & (dataset.target==3),'target'] = 20
dataset.loc[(dataset.Task.str.contains('R06')) & (dataset.target==2),'target'] = 30
dataset.loc[(dataset.Task.str.contains('R06')) & (dataset.target==3),'target'] = 40
dataset.loc[(dataset.Task.str.contains('R08')) & (dataset.target==2),'target'] = 10
dataset.loc[(dataset.Task.str.contains('R08')) & (dataset.target==3),'target'] = 20
dataset.loc[(dataset.Task.str.contains('R10')) & (dataset.target==2),'target'] = 30
dataset.loc[(dataset.Task.str.contains('R10')) & (dataset.target==3),'target'] = 40
dataset.loc[(dataset.Task.str.contains('R12')) & (dataset.target==2),'target'] = 10
dataset.loc[(dataset.Task.str.contains('R12')) & (dataset.target==3),'target'] = 20
dataset.loc[(dataset.Task.str.contains('R14')) & (dataset.target==2),'target'] = 30
dataset.loc[(dataset.Task.str.contains('R14')) & (dataset.target==3),'target'] = 40

In [7]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,632,633,634,635,636,637,638,639,target,Task
0,0.000004,-0.000014,-0.000040,-0.000055,-0.000071,-0.000067,-0.000071,-0.000052,-0.000039,-0.000054,...,-0.000026,-0.000014,-0.000016,-0.000025,-0.000016,-0.000035,-0.000054,-0.000054,0,S038R14
1,-0.000014,-0.000013,-0.000007,0.000010,0.000035,0.000006,-0.000001,0.000012,0.000005,0.000021,...,-0.000020,-0.000012,0.000003,0.000016,-0.000002,-0.000005,-0.000020,-0.000020,30,S038R14
2,-0.000008,-0.000006,-0.000002,-0.000007,-0.000004,0.000014,-0.000004,-0.000022,-0.000030,-0.000039,...,-0.000004,-0.000005,-0.000016,-0.000021,0.000003,0.000001,-0.000029,-0.000012,0,S038R14
3,-0.000027,-0.000036,-0.000026,-0.000021,-0.000008,-0.000012,-0.000010,-0.000004,-0.000004,-0.000015,...,-0.000075,-0.000043,-0.000032,-0.000047,-0.000004,0.000002,-0.000018,-0.000022,40,S038R14
4,0.000018,0.000022,0.000042,0.000052,0.000044,0.000015,-0.000031,0.000031,0.000097,0.000075,...,-0.000002,0.000010,0.000003,-0.000005,0.000009,0.000039,0.000013,0.000028,0,S038R14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19671,0.000116,0.000065,0.000086,0.000103,0.000062,0.000159,0.000125,0.000071,0.000057,-0.000009,...,-0.000200,-0.000194,-0.000210,-0.000244,-0.000250,-0.000185,-0.000260,-0.000253,20,S108R08
19672,-0.000153,-0.000158,-0.000166,-0.000215,-0.000242,-0.000217,-0.000170,-0.000212,-0.000222,-0.000183,...,0.000346,0.000452,0.000496,0.000459,0.000548,0.000600,0.000570,0.000557,0,S108R08
19673,0.000086,0.000041,-0.000060,-0.000122,-0.000157,-0.000053,-0.000198,-0.000231,-0.000256,-0.000204,...,-0.000062,-0.000078,-0.000098,-0.000145,-0.000076,-0.000122,-0.000062,-0.000084,10,S108R08
19674,-0.000021,-0.000012,-0.000025,-0.000087,-0.000043,0.000009,0.000026,-0.000017,-0.000026,-0.000090,...,0.000029,0.000035,0.000002,0.000066,-0.000001,0.000033,0.000052,0.000039,0,S108R08


In [8]:
dataset.to_csv('../dataset/DataSet.csv',index=False)